The existing run chart is as follows：

<p align="center">
  <img src="scheduling_graph_dijkstra.png" />
</p>

According to Dijkstra's algorithm, the steps to find the shortest path should be：


|Step|$\textit{dist}(H2, H0), \textit{pred}(H0)$|$\textit{dist}(H2, H1), \textit{pred}(H1)$|$\textit{dist}(H2, H3), \textit{pred}(H3)$|$\textit{dist}(H2, H4), \textit{pred}(H4)$|PriorityQueue|
|-------|--------------|--------------|--------------|--------------|--------------|
|$0$|$\infty$|$\infty$|$\infty$|$\infty$|$H2$|
|$1$|$\infty$|$\infty$|(9 - 10),$H2$|(8 - 9),$H2$|$H4$,$H3$|
|$2$|$\infty$|(8 - 9)(10 - 11),$H4$|(9 - 10),$H2$|-|$H3$,$H1$|
|$3$|(9 - 10)(15 - 15.5),$H3$|(8 - 9)(10 - 11),$H4$|-|-|$H1$,$H0$|
|$4$|(9 - 10)(15 - 15.5),$H3$|-|-|-|$H0$|


Algorithm implementation：

In [1]:
from typing import List, Tuple

def generate_graph(schedules:List[List[List[int]]]) -> List[List[Tuple[int, int, float]]]:
    """Function to generate a scheduling graph from a list of schedules. The scheduling graph is returned as
    adjacency lists [(destination, start_time, arrival_time)]. 
    In a graph of three nodes 0, 1 and 2 with the edges 0->1 from 8 to 10.42, 0->1 from 11 to 12,
    1->2 from 11 to 13.37 and 1->0 from 8 to 17, the respective representation would look as follows:
    [[(1, 8, 10.42), (1, 11, 12)], [(2, 11, 13.37), (0, 8, 17)]]"""
    tuple_tmp=Tuple[int, int, float]
    #   init of output: adjazenzliste
    adjazenzliste = []
    #   size of input list
    len_list = len(schedules)
    #  Iterate through each starting stations
    for start in range(len_list):                               
        list_tmp = []
        #   Iterate through each destination stations
        for destination in range(len_list): 
            #   Iterate through 24 hours                          
            for start_time in range(24): 
                #   Find the hour of start                            
                if schedules[start][destination][start_time] != float('inf'): 
                    #   Calculate arrival time     
                    arrival_time = start_time + schedules[start][destination][start_time]   
                    #   Give up cross-day schedule             
                    if arrival_time < 24:   
                        #   Record (destination, start_time, arrival_time) into tuple                           
                        tuple_tmp = (destination,start_time,arrival_time)  
                        #   Record tuple into list               
                        list_tmp.append(tuple_tmp)           
        # if len(out_tmp):
        #     adjazenzliste.append(out_tmp)
        adjazenzliste.append(list_tmp)                              
    return adjazenzliste


In [ ]:
from queue import PriorityQueue
from typing import List, Tuple

def scheduling_dijkstra(graph:List[List[Tuple[int, int, float]]], start:int, target:int, start_time:float) -> List[Tuple[int, float]]:
    """Implementation of Dijkstra's algorithm to determine a shortest path in the given scheduling graph. Takes a scheduling graph in 
    form of adjacency lists, a starting node, a starting time and a target node as its input and returns a shortest path from the 
    starting node to the target node. The path is given in the form of a list of tuples containing node and arrival time at that node
    in the path. A path which starts at node 0 at time 8, arrives at node 1 from node 0 at time 10.42 and reaches the target node 2
    at time 13.37 would be represented as [(0, 8), (1, 10.42), (2, 13.37)]."""    
    
    tuple_temp = ()
    station_num = len(graph)
    dist_arrival = []
    dic_path = {}
    visited = []
    q = PriorityQueue()
    curr = []
    path = []
    shortest_path = []

    for i in range(station_num):
        #   Set the initial arrival time of each node to 999
        dist_arrival.append(999)   
        #   The previous node of the initial path is None
        dic_path[i] = None      
        #   initial for all nodes not visited
        visited.append(0) 
    #   Set the arrival time of start node to 0   
    dist_arrival[start] = 0
    
    #   Iterate over the tuple of nodes next to the starting node
    for destination in graph[start]:
        #   Select possible edge by using start time
        if destination[1] >= start_time:
            #   Put start node into priority queue
            q.put((destination[2],destination)) 
            #   Set the arrival time of next node     
            dist_arrival[destination[0]] = destination[2]
            #   Record previous nodes
            dic_path[destination[0]]=start
    while not q.empty():
        #   Take the next node from the priority queue with the earliest arrival time
        tmp = q.get()
        #   The tuple of next node 
        curr = tmp[1]
        #   Check whether the node has been visited, if yes, do next while loop, otherwise run the next line of code
        if visited[curr[0]]:     
            continue
        #   Set visited node to 1
        visited[curr[0]] = 1
        #   Record the arrival time of this node
        dist_arrival[curr[0]] = tmp[0]
        #   Iterate over the next node of the current node
        for temp in graph[curr[0]]:
            #   the next node not been visited & the departure time is later than the arrival time of the current node 
            #   & the arrival time of next node is earlier than the time now recorded for next node
            if (not visited[temp[0]]) & (temp[2] <= dist_arrival[temp[0]]) & (tmp[0] <= temp[1]):
                #   Update arrival time, record parent node, put next node into priority queue
                dist_arrival[temp[0]] = temp[2]
                dic_path[temp[0]]=curr[0]           
                q.put((temp[2],temp)) 
    
    if dist_arrival[target] >= 24:
        #   If it is not possible to arrive today, give an information message 
        return print("Attention: Unable to arrive today")
    else:
        #   Output shortest path
        last = target
        #   Reverse path finding from the end node
        while True:
            tuple_temp = (last,dist_arrival[last])
            shortest_path.append(tuple_temp)
            last = dic_path[last]
            if last == start:
                break
        #   Find the second node from the starting node and the arrival time to the second node
        sec_node = shortest_path[-1][0]
        sec_arrival = shortest_path[-1][1]
        #   Using the arrival time and the second node number, traverse the neighboring nodes of the starting node
        #   to find the departure time (departure time should be later than the start time)
        for sec in graph[start]:
            if (sec[0] == sec_node) & (sec[-1] == sec_arrival) & (sec[1] >= start_time):
                depature_time = sec[1]
        #   Add the last tuple
        shortest_path.append((start,depature_time))
        #   Revert the path to start-end order
        shortest_path.reverse()
        return shortest_path



Test samples are shown below： **（If there is a next day arrival, a notification should be returned）**

In [ ]:
example = [[[float('inf')] * 24 for _ in range(3)] for _ in range(3)]

example[0][1][8] = 2.42
example[0][1][11] = 1

example[1][0][8] = 9
example[1][2][11] = 2.37

example_graph = generate_graph(example)
print(scheduling_dijkstra(example_graph, 0, 2, 6.66))

h1 = [[[float('inf')] * 24 for _ in range(5)] for _ in range(5)]

h1[2][4][8] = 1
h1[2][3][9] = 1

h1[4][2][8] = 1
h1[4][1][10] = 1
h1[4][1][19] = 3

h1[3][0][15] = 0.5

h1[1][3][14] = 0.5
h1[1][0][15] = 1.5
h1[1][3][23] = 2

h1[0][2][13] = 1
h1[0][1][13] = 1

graph1 = generate_graph(h1)
print(scheduling_dijkstra(graph1, 2, 0, 8))
print(scheduling_dijkstra(graph1, 2, 0, 12))

h2 = [[[float('inf')] * 24 for _ in range(4)] for _ in range(4)]

h2[0][1][4] = 1.5
h2[0][2][7] = 1
h2[0][3][9] = 8

h2[1][2][6] = 1.5
h2[1][3][6] = 6

h2[2][1][9] = 1
h2[2][3][9] = 2

h2[3][0][4] = 1

graph2 = generate_graph(h2)
print(scheduling_dijkstra(graph2, 0, 3, 2))

[(0, 8), (1, 10.42), (2, 13.370000000000001)]
[(2, 9), (3, 10), (0, 15.5)]
Attention: Unable to arrive today
None
[(0, 4), (1, 5.5), (2, 7.5), (3, 11)]
